In [1]:
%load_ext Cython

In [12]:
%%cython

import numpy as np
import pandas as pd
cimport numpy as cnp
import array

y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = np.concatenate([y, x], axis=1)

result_3 = np.zeros(5000)

cpdef double target_mean_v3(cnp.ndarray[long, ndim=2] data, cnp.ndarray[double, ndim=1] result_3):
    cdef int n = data.shape[0]
    cdef dict value = {}
    cdef dict count = {}
    #cdef cnp.ndarray[double, ndim=1] result = np.zeros(n) 
    
    # step1: 统计value和count

    for i in range(n):
      if data[i, 1] not in value.keys():
        value[data[i][1]] = data[i][0]
        count[data[i][1]] = 0

      else:
        value[data[i][1]] += data[i][0]
        count[data[i][1]] += 1

    # step2: 算出对应平均值
    for i in range(n):
      result_3[i] = (value[data[i, 1]] - data[i, 0])/ (count[data[i, 1]] - 1)

    #result_3 = result
    
    #print(value)
    #print(count)
    #print(result_3)

target_mean_v3(data, result_3)    

print(result_3)


[0.46837945 0.51515152 0.53846154 ... 0.49781659 0.46855984 0.4969574 ]


In [13]:
%%timeit
target_mean_v3(data, result_3)

100 loops, best of 3: 14.7 ms per loop


In [8]:
# coding = 'utf-8'
import numpy as np
import pandas as pd


def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    #print('groupby_result \n', groupby_result)
    #print('data[data.index != 1]', data[data.index != 1])
    return result

In [9]:
%%timeit
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

result_1 = target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 25.6 s per loop


In [10]:
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [11]:
%%timeit
y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

result_2 = target_mean_v2(data, 'y', 'x')

1 loop, best of 3: 276 ms per loop
